## Exercise 1

In [34]:
from astropy.constants import M_sun, M_earth, M_jup, G,c
from astropy.units import kg, yr, quantity_input, nm
from astropy.units.quantity import Quantity
import numpy as np
import pandas as pd
def format_tex(float_number):
    if isinstance(float_number, Quantity):
        float_number = float_number.to_value()
    if not isinstance(float_number, float):
        return float_number
    exponent = np.floor(np.log10(float_number))
    if -2 < exponent < 2:
        return f"{float_number:.4g}"
    mantissa = float_number/10**exponent
    mantissa_format = str(mantissa)[0:4]
    return "${0}\times10^{{{1}}}$"\
           .format(mantissa_format, str(int(exponent)))

@quantity_input(m_planet='mass', m_star='mass', period="time")
def calculate_max_vel(m_planet, m_star, period):
    frac = G*4*np.pi**2 / ((m_planet+ m_star)**2* period)
    return (m_planet * frac**(1/3)).to("m/s")

@quantity_input(delta_v="velocity")
def calculate_delta_h_alpha(delta_v, rest_wavelength=656.281 * nm):
    return (rest_wavelength* delta_v/c).to("nm")

M_sat = 5.68e26 * kg
P_earth = 1 *yr
P_jup = 11.86*yr
P_sat = 29.46*yr
# Construct dataframe:
df = pd.DataFrame({"Name": ["Earth", "Jupiter", "Saturn"], "Mass": [M_earth.to("kg"), M_jup.to("kg"), M_sat], "Period": [P_earth, P_jup, P_sat]})
df["Delta_v"] = df.apply(lambda obj: calculate_max_vel(obj["Mass"], M_sun, obj["Period"]), axis=1)
df["Delta_lambda"] = df["Delta_v"].apply(calculate_delta_h_alpha)
df = df.applymap(lambda x:format_tex(x))
print(df.to_latex(index=False, float_format="%.2f", escape=False))

\begin{tabular}{lllll}
\toprule
   Name &                Mass & Period & Delta_v &        Delta_lambda \\
\midrule
  Earth & $5.97\times10^{24}$ &      1 &  0.1651 & $3.61\times10^{-7}$ \\
Jupiter & $1.89\times10^{27}$ &  11.86 &   22.99 & $5.03\times10^{-5}$ \\
 Saturn & $5.68\times10^{26}$ &  29.46 &   5.082 & $1.11\times10^{-5}$ \\
\bottomrule
\end{tabular}



## Exercise 2

In [47]:
from astropy.constants import R_earth, R_jup, R_sun
from astropy.units import km, quantity_input, Quantity
import numpy as np
import pandas as pd

@quantity_input(r_obj="length")
def calculate_delta_mag(r_obj) -> Quantity:
    """Calculate the difference in magnitudes if the object with the
    radius r_obj would pass in front of the sun."""
    return -2.5 *np.log10(1 - r_obj**2/R_sun**2)

R_merc = 2439.7*km
R_nep = 24622 * km
# Construct dataframe:
df = pd.DataFrame({"Name": ["Mercury", "Earth", "Jupiter", "Neptune"], "Radius": [R_merc, R_earth, R_jup, R_nep]})
df["DM"] = df["Radius"].apply(calculate_delta_mag)
df = df.applymap(lambda x:format_tex(x))
print(df.to_latex(index=False, float_format="%.2f", escape=False))

\begin{tabular}{lll}
\toprule
   Name &             Radius &                  DM \\
\midrule
Mercury & $2.43\times10^{3}$ & $1.33\times10^{-5}$ \\
  Earth & $6.37\times10^{6}$ & $9.12\times10^{-5}$ \\
Jupiter & $7.14\times10^{7}$ & $1.15\times10^{-2}$ \\
Neptune & $2.46\times10^{4}$ & $1.36\times10^{-3}$ \\
\bottomrule
\end{tabular}



## Exercise 3

In [39]:
from astropy.constants import au
# Taken from https://de.wikipedia.org/wiki/Liste_der_Planeten_des_Sonnensystems
a_merc = 0.38709888 * au
a_earth = 1* au
a_jup = 5.2033623*au
a_nep = 30.068960 * au
df = pd.DataFrame({"Name": ["Mercury", "Earth", "Jupiter", "Neptune"], "a": [a_merc, a_earth, a_jup, a_nep]})
df["prob"] = df["a"].apply(lambda a: R_sun/a)
df = df.applymap(lambda x:format_tex(x))
print(df.to_latex(index=False, float_format="%.2f", escape=False))

\begin{tabular}{lll}
\toprule
   Name &                   a &                prob \\
\midrule
Mercury & $5.79\times10^{10}$ & $1.20\times10^{-2}$ \\
  Earth & $1.49\times10^{11}$ & $4.65\times10^{-3}$ \\
Jupiter & $7.78\times10^{11}$ & $8.93\times10^{-4}$ \\
Neptune & $4.49\times10^{12}$ & $1.54\times10^{-4}$ \\
\bottomrule
\end{tabular}



## Exercise 4

In [46]:
@quantity_input(r_star="length", m_star="mass", a_planet="length")
def calculate_transit_times(r_star, m_star, a_planet):
    return (2*r_star*(a_planet/(G*m_star))**0.5).to("h")

@quantity_input( m_star="mass", a_planet="length")
def calculate_orbital_period(m_star, a_planet):
    return ((4*np.pi**2*a_planet**3 / (G* m_star))**0.5).to("h")

print(calculate_transit_times(R_sun, M_sun, au))
a_merc = 0.38709888 * au
a_earth = 1* au
a_jup = 5.2033623*au
a_nep = 30.068960 * au
df = pd.DataFrame({"Name": ["Mercury", "Earth", "Jupiter", "Neptune"], "a": [a_merc, a_earth, a_jup, a_nep]})
df["t [h]"] = df["a"].apply(lambda a: calculate_transit_times(R_sun, M_sun, a))
df["Period [yr]"] = df["a"].apply(lambda a: calculate_orbital_period(M_sun, a))
df["Period_frac"] = df["t [h]"]/df["Period [h]"]

df = df.applymap(lambda x:format_tex(x))
print(df.to_latex(index=False, float_format="%.2f", escape=False))

12.976464628547822 h
\begin{tabular}{lllll}
\toprule
   Name &                   a & t [h] &         Period [h] &         Period_frac \\
\midrule
Mercury & $5.79\times10^{10}$ & 8.074 & $2.11\times10^{3}$ & $3.82\times10^{-3}$ \\
  Earth & $1.49\times10^{11}$ & 12.98 & $8.76\times10^{3}$ & $1.48\times10^{-3}$ \\
Jupiter & $7.78\times10^{11}$ &  29.6 & $1.04\times10^{5}$ & $2.84\times10^{-4}$ \\
Neptune & $4.49\times10^{12}$ & 71.16 & $1.44\times10^{6}$ & $4.92\times10^{-5}$ \\
\bottomrule
\end{tabular}



## Exercise 6

In [39]:
import pandas as pd
import numpy as np




names = np.array(["H", "He", "O", "Mg", "Si", "Fe"])
relative_numbers = np.array([1, 8e-2, 7e-4, 4e-5, 4e-5, 3e-5])
weights = np.array([1, 4, 16, 24.3, 28.1, 55.8])
# df = pd.DataFrame(np.array([names, relative_numbers, weights]).T, columns=["Name", "Rel", "Weight"])
# df.set_index("Name", inplace=True)
# df.T
df = pd.DataFrame([relative_numbers, weights], columns=names, index=["Num", "Weight"])
df.head()

Mg_2_Si_O_4 = df["H"]["Weight"]*2+ df["Si"]["Weight"] + df["O"]["Weight"]
Mg_2_Si_O_4
print(df.to_latex(index=True, escape=False, float_format="%.3g"))

\begin{tabular}{lrrrrrr}
\toprule
{} &  H &   He &      O &    Mg &    Si &    Fe \\
\midrule
Num    &  1 & 0.08 & 0.0007 & 4e-05 & 4e-05 & 3e-05 \\
Weight &  1 &    4 &     16 &  24.3 &  28.1 &  55.8 \\
\bottomrule
\end{tabular}

